# Sistema de Arbitraje HFT - Pipeline Completo

## Índice

1. [Introducción](#introduccion)
2. [Configuración y Selección de Dataset](#configuracion)
3. [Fase 1: Carga de Datos](#fase1)
4. [Fase 2: Limpieza de Datos](#fase2)
5. [Fase 3: Consolidated Tape](#fase3)
6. [Fase 4: Detección de Señales](#fase4)
7. [Fase 5: Ejecución Instantánea](#fase5)
8. [Fase 6: Análisis Final](#fase6)
9. [Visualizaciones y Reportes](#visualizaciones)

---

## [WARNING] IMPORTANTE: Selección de Dataset

**Antes de ejecutar el notebook**, asegúrate de configurar el dataset en la **celda 1**:

- **DATA_SMALL**: Para pruebas rápidas (más rápido)
- **DATA_BIG**: Para análisis completo (más datos, más lento)

Cambia la variable `USE_DATA_BIG = False` a `True` si quieres usar DATA_BIG.

---

## Introducción {#introduccion}

Este notebook implementa un sistema completo de detección de arbitraje en mercados fragmentados europeos.

### La Analogía del Mercado de Frutas

Imagina que tienes **4 mercados de frutas** (XMAD, AQXE, CEUX, TRQX) donde se venden manzanas.

En cada mercado, en cada momento, hay:
- Un **VENDEDOR** con el precio **MÁS BAJO** al que está dispuesto a vender (ASK)
- Un **COMPRADOR** con el precio **MÁS ALTO** al que está dispuesto a comprar (BID)

### La Regla de Oro

**UNA OPORTUNIDAD EXISTE CUANDO:**

```
MAX(todos los bids) > MIN(todos los asks)
```

Es decir:
- El precio **MÁS ALTO** que alguien está dispuesto a **PAGAR** en CUALQUIER mercado
- Es **MAYOR** que
- El precio **MÁS BAJO** al que alguien está dispuesto a **VENDER** en CUALQUIER mercado

Cuando esto pasa → **ARBITRAJE POSIBLE** [OK]

### Ejemplo Práctico

```
Mercado XMAD:  Comprador ofrece 10.52€ | Vendedor pide 10.54€
Mercado AQXE:  Comprador ofrece 10.49€ | Vendedor pide 10.51€
Mercado CEUX:  Comprador ofrece 10.48€ | Vendedor pide 10.53€
Mercado TRQX:  Comprador ofrece 10.50€ | Vendedor pide 10.52€
```

**Análisis:**
- **Mejor comprador:** XMAD (10.52€) ← El que más paga
- **Mejor vendedor:** AQXE (10.51€) ← El que menos pide

**¡OPORTUNIDAD!** 10.52€ > 10.51€

**Tu arbitraje:**
1. Compras en AQXE por 10.51€
2. Vendes en XMAD por 10.52€
3. Ganancia: 0.01€ por manzana

Si hay 300 manzanas disponibles → Ganancia total: **3.00€**

---

## Configuración {#configuracion}

### Asunciones del Modelo

- [OK] **Latencia = 0** (ejecución instantánea)
- [OK] **Sin comisiones** de mercado
- [OK] **Profit teórico = Profit real**

### Estructura del Pipeline

```
┌─────────────────┐
│  1. CARGA       │  → Leer archivos QTE y STS
└────────┬────────┘
         │
┌────────▼────────┐
│  2. LIMPIEZA    │  → Eliminar magic numbers, filtrar por status
└────────┬────────┘
         │
┌────────▼────────┐
│  3. CONSOLIDAR  │  → Crear tape único con todos los venues
└────────┬────────┘
         │
┌────────▼────────┐
│  4. DETECTAR    │  → Buscar MAX(bid) > MIN(ask)
└────────┬────────┘
         │
┌────────▼────────┐
│  5. EJECUTAR    │  → Calcular profit (latencia=0)
└────────┬────────┘
         │
┌────────▼────────┐
│  6. ANALIZAR    │  → Generar reportes y visualizaciones
└─────────────────┘
```


In [18]:
# ============================================================================
# IMPORTS Y CONFIGURACIÓN INICIAL
# ============================================================================

import sys
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import logging
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configurar paths
PROJECT_ROOT = Path.cwd()
sys.path.insert(0, str(PROJECT_ROOT / 'src'))

# Configurar logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

print("[OK] Imports completados")
print(f"[INFO] Directorio de trabajo: {PROJECT_ROOT}")

# ============================================================================
# SELECTOR DE DATASET
# ============================================================================
print("\n" + "=" * 80)
print("SELECCIÓN DE DATASET")
print("=" * 80)

# Función helper para display_dataframe (igual que en main_script.py)
def display_dataframe(df: pd.DataFrame, title: str = "", max_rows: int = 20):
    """Muestra un DataFrame de forma clara y legible."""
    if df is None or len(df) == 0:
        print(f"\n  {title}: (vacío)")
        return
    
    print(f"\n  {'=' * 76}")
    if title:
        print(f"  {title}")
        print(f"  {'=' * 76}")
    
    display_df = df.head(max_rows)
    
    with pd.option_context('display.max_columns', None,
                          'display.width', None,
                          'display.max_colwidth', 50):
        print(display_df.to_string(index=False))
    
    if len(df) > max_rows:
        print(f"\n  ... (mostrando {max_rows} de {len(df)} filas totales)")
    
    print(f"  {'=' * 76}")

# Selección interactiva del dataset
print("\n[INFO] Selecciona el dataset a usar:")
print("   1. DATA_SMALL (rápido, para testing)")
print("   2. DATA_BIG (completo, para producción)")

# Por defecto usar DATA_SMALL, pero se puede cambiar manualmente
USE_DATA_BIG = False  # Cambiar a True para usar DATA_BIG

if USE_DATA_BIG:
    print("\n[OK] Seleccionado: DATA_BIG")
    DATA_DIR_NAME = "DATA_BIG"
else:
    print("\n[OK] Seleccionado: DATA_SMALL")
    DATA_DIR_NAME = "DATA_SMALL"

print(f"[INFO] Dataset seleccionado: {DATA_DIR_NAME}")
print("=" * 80)


[OK] Imports completados
[INFO] Directorio de trabajo: c:\Users\Pc\Downloads\TAREA_RENTA_VARIABLE

SELECCIÓN DE DATASET

[INFO] Selecciona el dataset a usar:
   1. DATA_SMALL (rápido, para testing)
   2. DATA_BIG (completo, para producción)

[OK] Seleccionado: DATA_SMALL
[INFO] Dataset seleccionado: DATA_SMALL


## Importar Módulos del Sistema

A continuación importamos todos los módulos necesarios:


In [19]:
# Importar todos los módulos del sistema
from config_module import config
from data_loader_module import DataLoader
from data_cleaner_module import DataCleaner
from consolidator_module import ConsolidatedTape
from signal_generator_module import SignalGenerator
from analyzer_module import ArbitrageAnalyzer

print("[OK] Todos los módulos importados correctamente")

# Configurar directorio de datos según selección
if DATA_DIR_NAME == "DATA_BIG":
    data_dir = config.DATA_BIG_DIR
else:
    data_dir = config.DATA_SMALL_DIR

print(f"\n[INFO] Configuración:")
print(f"   - Dataset seleccionado: {DATA_DIR_NAME}")
print(f"   - Directorio de datos: {data_dir}")
print(f"   - Directorio de output: {config.OUTPUT_DIR}")
print(f"   - Magic numbers a filtrar: {len(config.MAGIC_NUMBERS)} valores")
print(f"   - Venues configurados: {list(config.VALID_STATES.keys())}")

# Verificar que el directorio existe
if not data_dir.exists():
    print(f"\n[ERROR] Directorio no encontrado: {data_dir}")
    print("Por favor, asegúrate de que los datos están en la ubicación correcta")
    raise FileNotFoundError(f"Directorio no encontrado: {data_dir}")

# Limpiar outputs anteriores (igual que en main_script.py)
import shutil
print("\n[INFO] Limpiando outputs anteriores...")
if config.OUTPUT_DIR.exists():
    for item in config.OUTPUT_DIR.iterdir():
        try:
            if item.is_file():
                item.unlink()
            elif item.is_dir():
                shutil.rmtree(item)
        except Exception as e:
            print(f"  No se pudo eliminar {item}: {e}")

config.OUTPUT_DIR.mkdir(exist_ok=True)
config.FIGURES_DIR.mkdir(exist_ok=True)
print("[OK] Directorios de output listos")


[OK] Todos los módulos importados correctamente

[INFO] Configuración:
   - Dataset seleccionado: DATA_SMALL
   - Directorio de datos: c:\Users\Pc\Downloads\TAREA_RENTA_VARIABLE\data\DATA_SMALL
   - Directorio de output: c:\Users\Pc\Downloads\TAREA_RENTA_VARIABLE\output
   - Magic numbers a filtrar: 6 valores
   - Venues configurados: ['XMAD', 'AQXE', 'AQEU', 'CEUX', 'TRQX', 'TQEX']

[INFO] Limpiando outputs anteriores...
[OK] Directorios de output listos


---

## FASE 1: CARGA DE DATOS {#fase1}

### Objetivo
Cargar archivos QTE (quotes) y STS (status) desde archivos comprimidos `.csv.gz`.

### Convención de Nombres
```
<type>_<session>_<isin>_<ticker>_<mic>_<part>.csv.gz

Ejemplo:
QTE_2024-01-15_ES0113900J37_SAN_XMAD_1.csv.gz
STS_2024-01-15_ES0113900J37_SAN_XMAD_1.csv.gz
```

### Book Identity Key
Cada order book se identifica únicamente por:
```
(session, isin, mic, ticker)
```

### Columnas Requeridas

**QTE (Quotes):**
- `epoch` (int64) - Timestamp en nanosegundos UTC
- `px_bid_0`, `px_ask_0` (float64) - Precios best bid/ask
- `qty_bid_0`, `qty_ask_0` (float64) - Cantidades disponibles
- Niveles 1-9 opcionales si existen

**STS (Status):**
- `epoch` (int64) - Timestamp en nanosegundos UTC
- `market_trading_status` (int64) - Código de estado del mercado

### Diagrama de Flujo

```
┌─────────────────────────────────────┐
│  Descubrir ISINs disponibles        │
└──────────────┬──────────────────────┘
               │
┌──────────────▼──────────────────────┐
│  Para cada ISIN:                     │
│  ┌──────────────────────────────┐  │
│  │ Buscar archivos QTE y STS     │  │
│  │ para cada venue (XMAD, AQXE,  │  │
│  │ CEUX, TRQX)                    │  │
│  └──────────────┬─────────────────┘  │
│                 │                     │
│  ┌──────────────▼─────────────────┐  │
│  │ Leer CSV comprimido             │  │
│  │ - Encoding: utf-8, latin-1      │  │
│  │ - Separador: ;                  │  │
│  │ - Decimal: .                    │  │
│  └──────────────┬─────────────────┘  │
│                 │                     │
│  ┌──────────────▼─────────────────┐  │
│  │ Validar columnas requeridas     │  │
│  │ - epoch debe ser int64          │  │
│  │ - Precios y cantidades float64  │  │
│  └──────────────┬─────────────────┘  │
│                 │                     │
│  ┌──────────────▼─────────────────┐  │
│  │ Retornar Dict[mic] ->          │  │
│  │   {'qte': DataFrame,            │  │
│  │    'sts': DataFrame}            │  │
│  └────────────────────────────────┘  │
└──────────────────────────────────────┘
```


In [20]:
# ============================================================================
# FASE 1: CARGA DE DATOS
# ============================================================================

print("\n" + "=" * 80)
print("FASE 1: CARGA DE DATOS")
print("=" * 80)

# Inicializar loader con el directorio seleccionado
loader = DataLoader(data_dir)
logger = logging.getLogger(__name__)
logger.info(f"Directorio de datos: {data_dir}")

# Descubrir ISINs disponibles
print("\n[INFO] Descubriendo ISINs disponibles...")
isins = loader.discover_isins()

if len(isins) == 0:
    logger.error("No se encontraron ISINs en el directorio")
    raise ValueError("No hay datos disponibles")

# Seleccionar primer ISIN para análisis
test_isin = isins[0]
logger.info(f"Analizando ISIN: {test_isin}")
print(f"\n[INFO] ISIN seleccionado: {test_isin}")

# Cargar datos raw
print(f"\n[INFO] Cargando datos para {test_isin}...")
raw_data = loader.load_isin_data(test_isin)

if len(raw_data) == 0:
    logger.error(f"No se pudieron cargar datos para {test_isin}")
    raise ValueError("Error en carga de datos")

# Mostrar resumen detallado de carga
print(f"\n[OK] Datos cargados exitosamente:")
print(f"   Total venues: {len(raw_data)}")
for mic, venue_data in raw_data.items():
    qte_df = venue_data.get('qte', pd.DataFrame())
    sts_df = venue_data.get('sts', pd.DataFrame())
    qte_rows = len(qte_df)
    sts_rows = len(sts_df)
    print(f"\n   [INFO] {mic}:")
    print(f"      - QTE rows: {qte_rows:,}")
    print(f"      - STS rows: {sts_rows:,}")
    if qte_rows > 0:
        print(f"      - Epoch range QTE: {qte_df['epoch'].min():,} - {qte_df['epoch'].max():,}")
    if sts_rows > 0:
        print(f"      - Epoch range STS: {sts_df['epoch'].min():,} - {sts_df['epoch'].max():,}")

print(f"\n[INFO] Resumen: {len(raw_data)} venues cargados exitosamente")


2025-12-09 14:23:35,121 - data_loader_module - INFO - DataLoader initialized: c:\Users\Pc\Downloads\TAREA_RENTA_VARIABLE\data\DATA_SMALL
2025-12-09 14:23:35,127 - __main__ - INFO - Directorio de datos: c:\Users\Pc\Downloads\TAREA_RENTA_VARIABLE\data\DATA_SMALL
2025-12-09 14:23:35,141 - __main__ - INFO - Analizando ISIN: ES0113900J37



FASE 1: CARGA DE DATOS

[INFO] Descubriendo ISINs disponibles...

DESCUBRIENDO ISINs DISPONIBLES
 Encontrados 1 ISINs unicos
  Primeros 5: ['ES0113900J37']

[INFO] ISIN seleccionado: ES0113900J37

[INFO] Cargando datos para ES0113900J37...

CARGANDO DATOS PARA ISIN: ES0113900J37
  Archivos QTE encontrados: 4

  [PROCESANDO] Venue: AQEU


2025-12-09 14:23:36,634 - data_loader_module - INFO -   [OK] QTE_2025-11-07_ES0113900J37_SANe_AQEU_1.csv.gz: 101,610 filas válidas (de 101,616)
2025-12-09 14:23:36,650 - data_loader_module - INFO -   [OK] STS_2025-11-07_ES0113900J37_SANe_AQEU_1.csv.gz: 6 filas
2025-12-09 14:23:36,693 - data_loader_module - INFO -   [DEBUG] AQEU: Columnas disponibles en QTE: ['session', 'inst_id', 'sequence', 'isin', 'ticker', 'mic', 'currency', 'epoch', 'event_timestamp', 'bloombergTicker', 'ord_bid_0', 'qty_bid_0', 'px_bid_0', 'px_ask_0', 'qty_ask_0', 'ord_ask_0', 'ord_bid_1', 'qty_bid_1', 'px_bid_1', 'px_ask_1']
2025-12-09 14:23:36,694 - data_loader_module - INFO -   [DEBUG] AQEU: Total columnas: 70
2025-12-09 14:23:36,695 - data_loader_module - INFO -   [DEBUG] AQEU: Filas en QTE: 101610


    [OK] AQEU: 101,610 snapshots

  [PROCESANDO] Venue: XMAD


2025-12-09 14:23:40,727 - data_loader_module - INFO -   [OK] QTE_2025-11-07_ES0113900J37_SAN_XMAD_1.csv.gz: 364,903 filas válidas (de 364,912)
2025-12-09 14:23:40,769 - data_loader_module - INFO -   [OK] STS_2025-11-07_ES0113900J37_SAN_XMAD_1.csv.gz: 7 filas
2025-12-09 14:23:40,867 - data_loader_module - INFO -   [DEBUG] XMAD: Columnas disponibles en QTE: ['session', 'inst_id', 'sequence', 'isin', 'ticker', 'mic', 'currency', 'epoch', 'event_timestamp', 'bloombergTicker', 'ord_bid_0', 'qty_bid_0', 'px_bid_0', 'px_ask_0', 'qty_ask_0', 'ord_ask_0', 'ord_bid_1', 'qty_bid_1', 'px_bid_1', 'px_ask_1']
2025-12-09 14:23:40,868 - data_loader_module - INFO -   [DEBUG] XMAD: Total columnas: 70
2025-12-09 14:23:40,870 - data_loader_module - INFO -   [DEBUG] XMAD: Filas en QTE: 364903


    [OK] XMAD: 364,903 snapshots

  [PROCESANDO] Venue: CEUX


2025-12-09 14:23:41,654 - data_loader_module - INFO -   [OK] QTE_2025-11-07_ES0113900J37_SANe_CEUX_1.csv.gz: 78,462 filas válidas (de 78,472)
2025-12-09 14:23:41,662 - data_loader_module - INFO -   [OK] STS_2025-11-07_ES0113900J37_SANe_CEUX_1.csv.gz: 7 filas
2025-12-09 14:23:41,686 - data_loader_module - INFO -   [DEBUG] CEUX: Columnas disponibles en QTE: ['session', 'inst_id', 'sequence', 'isin', 'ticker', 'mic', 'currency', 'epoch', 'event_timestamp', 'bloombergTicker', 'ord_bid_0', 'qty_bid_0', 'px_bid_0', 'px_ask_0', 'qty_ask_0', 'ord_ask_0', 'ord_bid_1', 'qty_bid_1', 'px_bid_1', 'px_ask_1']
2025-12-09 14:23:41,687 - data_loader_module - INFO -   [DEBUG] CEUX: Total columnas: 70
2025-12-09 14:23:41,688 - data_loader_module - INFO -   [DEBUG] CEUX: Filas en QTE: 78462


    [OK] CEUX: 78,462 snapshots

  [PROCESANDO] Venue: TQEX


2025-12-09 14:23:42,536 - data_loader_module - INFO -   [OK] QTE_2025-11-07_ES0113900J37_SANe_TQEX_1.csv.gz: 43,316 filas válidas (de 43,325)
2025-12-09 14:23:42,545 - data_loader_module - INFO -   [OK] STS_2025-11-07_ES0113900J37_SANe_TQEX_1.csv.gz: 3 filas
2025-12-09 14:23:42,560 - data_loader_module - INFO -   [DEBUG] TQEX: Columnas disponibles en QTE: ['session', 'inst_id', 'sequence', 'isin', 'ticker', 'mic', 'currency', 'epoch', 'event_timestamp', 'bloombergTicker', 'ord_bid_0', 'qty_bid_0', 'px_bid_0', 'px_ask_0', 'qty_ask_0', 'ord_ask_0', 'ord_bid_1', 'qty_bid_1', 'px_bid_1', 'px_ask_1']
2025-12-09 14:23:42,561 - data_loader_module - INFO -   [DEBUG] TQEX: Total columnas: 70
2025-12-09 14:23:42,561 - data_loader_module - INFO -   [DEBUG] TQEX: Filas en QTE: 43316


    [OK] TQEX: 43,316 snapshots

[EXITO] Venues cargados: ['AQEU', 'XMAD', 'CEUX', 'TQEX']

[OK] Datos cargados exitosamente:
   Total venues: 4

   [INFO] AQEU:
      - QTE rows: 101,610
      - STS rows: 6
      - Epoch range QTE: 1,762,502,416,697,531 - 1,762,533,000,141,495
      - Epoch range STS: 1,762,500,600,000,015 - 1,762,533,156,000,006

   [INFO] XMAD:
      - QTE rows: 364,903
      - STS rows: 7
      - Epoch range QTE: 1,762,495,202,094,554 - 1,762,534,870,304,556
      - Epoch range STS: 1,762,495,202,277,854 - 1,762,533,900,424,604

   [INFO] CEUX:
      - QTE rows: 78,462
      - STS rows: 7
      - Epoch range QTE: 1,762,502,416,697,549 - 1,762,533,375,093,153
      - Epoch range STS: 1,762,494,604,118,987 - 1,762,538,400,000,017

   [INFO] TQEX:
      - QTE rows: 43,316
      - STS rows: 3
      - Epoch range QTE: 1,762,502,416,697,156 - 1,762,533,001,057,722
      - Epoch range STS: 1,762,502,399,028,046 - 1,762,533,901,374,607

[INFO] Resumen: 4 venues cargados ex

---

## FASE 2: LIMPIEZA DE DATOS {#fase2}

### Objetivo
Eliminar datos inválidos y filtrar por estado de mercado.

### Pipeline de Limpieza (ORDEN CRÍTICO)

```
┌─────────────────────────────────────┐
│  PASO 1: Eliminar Magic Numbers     │
│  ─────────────────────────────────  │
│  Magic numbers NO son precios reales │
│  • 666666.666 → Unquoted/Unknown    │
│  • 999999.999 → Market Order        │
│  • 999999.989 → At Open Order       │
│  • 999999.988 → At Close Order      │
│  • 999999.979 → Pegged Order        │
│  • 999999.123 → Unquoted/Unknown    │
└──────────────┬──────────────────────┘
               │
┌──────────────▼──────────────────────┐
│  PASO 2: Filtrar por Trading Status │
│  ─────────────────────────────────  │
│  Solo Continuous Trading es válido:  │
│  • XMAD: [5832713, 5832756]         │
│  • AQXE: [5308427]                   │
│  • CEUX: [12255233]                  │
│  • TRQX: [7608181]                   │
│                                      │
│  Usa merge_asof con direction=       │
│  'backward' para propagar el último │
│  estado conocido a cada quote       │
└──────────────┬──────────────────────┘
               │
┌──────────────▼──────────────────────┐
│  PASO 3: Validar Precios            │
│  ─────────────────────────────────  │
│  • px_bid_0 > 0 y px_ask_0 > 0      │
│  • px_bid_0 < px_ask_0 (no crossed) │
│  • qty_bid_0 > 0 y qty_ask_0 > 0    │
│  • Precios < 10000 EUR (sanity)     │
└─────────────────────────────────────┘
```

### Validación del Book Identity Key

Antes de hacer merge QTE-STS, verificamos que pertenecen al mismo order book:

```
Book Key = (session, isin, mic, ticker)

Si QTE.session ≠ STS.session → ERROR
Si QTE.isin ≠ STS.isin → ERROR
Si QTE.ticker ≠ STS.ticker → ERROR
```

Esto previene joins incorrectos entre diferentes instrumentos.


In [21]:
# ============================================================================
# FASE 2: LIMPIEZA DE DATOS
# ============================================================================

print("\n" + "=" * 80)
print("FASE 2: LIMPIEZA Y VALIDACIÓN")
print("=" * 80)

# Inicializar cleaner
cleaner = DataCleaner()

# Aplicar limpieza a todos los venues
print("\n[INFO] Aplicando pipeline de limpieza...")
clean_data = cleaner.clean_all_venues(raw_data)

if len(clean_data) == 0:
    print("[ERROR] No quedan datos después de la limpieza")
    raise ValueError("Todos los datos fueron eliminados en la limpieza")

# Mostrar resumen de limpieza
print(f"\n[OK] Limpieza completada:")
for mic in clean_data.keys():
    if mic in raw_data:
        original_qte = len(raw_data[mic].get('qte', pd.DataFrame()))
        cleaned_qte = len(clean_data[mic])
        retention = (cleaned_qte / original_qte * 100) if original_qte > 0 else 0
        print(f"   {mic}: {cleaned_qte:,} / {original_qte:,} rows ({retention:.2f}% retenido)")

print(f"\n[INFO] Venues válidos después de limpieza: {len(clean_data)}")


2025-12-09 14:23:42,799 - data_cleaner_module - INFO -     Códigos esperados para AQEU: [5308427]
2025-12-09 14:23:42,800 - data_cleaner_module - INFO -     Códigos encontrados en STS: [np.int64(5308426), np.int64(5308427), np.int64(5308428), np.int64(5308429)]
2025-12-09 14:23:42,801 - data_cleaner_module - INFO -     Códigos que coinciden: [5308427]



FASE 2: LIMPIEZA Y VALIDACIÓN

[INFO] Aplicando pipeline de limpieza...

LIMPIEZA Y VALIDACIÓN DE DATOS

  [LIMPIEZA] AQEU...
    Snapshots iniciales: 101,610


2025-12-09 14:23:42,867 - data_cleaner_module - INFO -     Snapshots con estado asignado: 101,610 (100.00%)
2025-12-09 14:23:42,868 - data_cleaner_module - INFO -     Snapshots sin estado asignado: 0 (0.00%)
2025-12-09 14:23:43,080 - data_cleaner_module - INFO -     Distribución de estados encontrados:
2025-12-09 14:23:43,082 - data_cleaner_module - INFO -       5308427: 101,559 snapshots ([VALID])
2025-12-09 14:23:43,083 - data_cleaner_module - INFO -       5308428: 51 snapshots ([INVALID])
2025-12-09 14:23:43,137 - data_cleaner_module - INFO -     [OK] Removed 51 non-trading snapshots (0.05%)
2025-12-09 14:23:43,138 - data_cleaner_module - INFO -     [OK] Kept 101,559 continuous trading snapshots (99.95%)


    [OK] Snapshots finales: 101,559 (99.95% retenido)

  [LIMPIEZA] XMAD...
    Snapshots iniciales: 364,903


2025-12-09 14:23:43,786 - data_cleaner_module - INFO -     Removed 1 magic numbers (0.00%)
2025-12-09 14:23:43,790 - data_cleaner_module - INFO -     Códigos esperados para XMAD: [5832713, 5832756]
2025-12-09 14:23:43,791 - data_cleaner_module - INFO -     Códigos encontrados en STS: [np.int64(5832754), np.int64(5832755), np.int64(5832756), np.int64(5832757), np.int64(5832758), np.int64(5832762), np.int64(5832763)]
2025-12-09 14:23:43,795 - data_cleaner_module - INFO -     Códigos que coinciden: [5832756]
2025-12-09 14:23:43,957 - data_cleaner_module - INFO -     Snapshots con estado asignado: 364,901 (100.00%)
2025-12-09 14:23:43,958 - data_cleaner_module - INFO -     Snapshots sin estado asignado: 1 (0.00%)
2025-12-09 14:23:45,496 - data_cleaner_module - INFO -     Distribución de estados encontrados:
2025-12-09 14:23:45,503 - data_cleaner_module - INFO -       5832756: 362,896 snapshots ([VALID])
2025-12-09 14:23:45,506 - data_cleaner_module - INFO -       5832757: 1,393 snapshots (

    [OK] Snapshots finales: 362,894 (99.45% retenido)

  [LIMPIEZA] CEUX...
    Snapshots iniciales: 78,462


2025-12-09 14:23:47,052 - data_cleaner_module - INFO -     [OK] Removed 87 non-trading snapshots (0.11%)
2025-12-09 14:23:47,054 - data_cleaner_module - INFO -     [OK] Kept 78,375 continuous trading snapshots (99.89%)
2025-12-09 14:23:47,234 - data_cleaner_module - INFO -     Códigos esperados para TQEX: [7608181]
2025-12-09 14:23:47,236 - data_cleaner_module - INFO -     Códigos encontrados en STS: [np.int64(7608181), np.int64(7608182), np.int64(7608183)]
2025-12-09 14:23:47,237 - data_cleaner_module - INFO -     Códigos que coinciden: [7608181]
2025-12-09 14:23:47,256 - data_cleaner_module - INFO -     Snapshots con estado asignado: 43,316 (100.00%)
2025-12-09 14:23:47,256 - data_cleaner_module - INFO -     Snapshots sin estado asignado: 0 (0.00%)
2025-12-09 14:23:47,298 - data_cleaner_module - INFO -     Distribución de estados encontrados:
2025-12-09 14:23:47,301 - data_cleaner_module - INFO -       7608181: 43,316 snapshots ([VALID])
2025-12-09 14:23:47,400 - data_cleaner_module 

    [OK] Snapshots finales: 78,375 (99.89% retenido)

  [LIMPIEZA] TQEX...
    Snapshots iniciales: 43,316
    [OK] Snapshots finales: 43,316 (100.00% retenido)

  [MÉTRICAS DE CALIDAD AGREGADAS]
    Filas originales: 588,291
    Eliminadas por magic numbers: 1 (0.00%)
    Eliminadas por status inválido: 2,144 (0.36%)
    Eliminadas por validaciones: 2 (0.00%)
    Filas finales: 586,144 (99.64% retenido)

[EXITO] Limpieza completada para 4 venues

[OK] Limpieza completada:
   AQEU: 101,559 / 101,610 rows (99.95% retenido)
   XMAD: 362,894 / 364,903 rows (99.45% retenido)
   CEUX: 78,375 / 78,462 rows (99.89% retenido)
   TQEX: 43,316 / 43,316 rows (100.00% retenido)

[INFO] Venues válidos después de limpieza: 4


---

## FASE 3: CONSOLIDATED TAPE {#fase3}

### Objetivo
Crear un DataFrame único donde cada fila es un instante de tiempo y las columnas contienen los precios de TODOS los venues simultáneamente.

### Estructura del Consolidated Tape

```
| epoch | XMAD_bid | XMAD_ask | XMAD_bid_qty | XMAD_ask_qty |
|       | AQXE_bid | AQXE_ask | AQXE_bid_qty | AQXE_ask_qty |
|       | CEUX_bid | CEUX_ask | CEUX_bid_qty | CEUX_ask_qty |
|       | TRQX_bid | TRQX_ask | TRQX_bid_qty | TRQX_ask_qty |
```

### Algoritmo

```
┌─────────────────────────────────────┐
│  PASO 1: Renombrar columnas         │
│  px_bid_0 → {MIC}_bid               │
│  px_ask_0 → {MIC}_ask               │
│  qty_bid_0 → {MIC}_bid_qty          │
│  qty_ask_0 → {MIC}_ask_qty          │
└──────────────┬──────────────────────┘
               │
┌──────────────▼──────────────────────┐
│  PASO 2: Merge iterativo             │
│  ─────────────────────────────────  │
│  • Empezar con primer venue          │
│  • Outer merge con cada venue        │
│  • Usar 'epoch' como key             │
│  • Resultado: todos los timestamps   │
│    únicos de todos los venues        │
└──────────────┬──────────────────────┘
               │
┌──────────────▼──────────────────────┐
│  PASO 3: Ordenar por timestamp      │
│  sort_values('epoch')               │
└──────────────┬──────────────────────┘
               │
┌──────────────▼──────────────────────┐
│  PASO 4: Forward Fill (CRÍTICO)      │
│  ─────────────────────────────────  │
│  Asunción de market microstructure: │
│  El último precio conocido sigue    │
│  vigente hasta que llegue un update  │
│                                      │
│  Ejemplo:                            │
│  XMAD actualiza en T=100 y T=200    │
│  → Precio en T=150 = precio de T=100│
└──────────────┬──────────────────────┘
               │
┌──────────────▼──────────────────────┐
│  PASO 5: Eliminar filas iniciales    │
│  con NaNs (si todas las columnas    │
│  son NaN)                            │
└─────────────────────────────────────┘
```

### ¿Por qué Forward Fill?

En mercados fragmentados, cada venue actualiza a diferentes frecuencias:
- XMAD puede actualizar cada 100ms
- AQXE puede actualizar cada 200ms
- CEUX puede actualizar cada 150ms

Sin forward fill, tendríamos NaNs en cada timestamp donde un venue no actualiza, lo cual haría **imposible comparar precios** entre venues.

Con forward fill, asumimos que el último precio conocido sigue vigente hasta el próximo update, que es estándar en análisis de order books.


In [22]:
# ============================================================================
# FASE 3: CONSOLIDATED TAPE
# ============================================================================

print("\n" + "=" * 80)
print("FASE 3: CONSOLIDATED TAPE")
print("=" * 80)

# Crear consolidador (con redondeo temporal opcional para datasets grandes)
tape_builder = ConsolidatedTape(time_bin_ms=100)  # 100ms bins

print("\n[INFO] Consolidando datos de todos los venues...")
consolidated_tape = tape_builder.create_tape(clean_data)

if consolidated_tape is None or len(consolidated_tape) == 0:
    print("[ERROR] Consolidated tape vacío o nulo")
    raise ValueError("Error creando consolidated tape")

# Validar tape
print("\n[OK] Validando consolidated tape...")
is_valid = tape_builder.validate_tape(consolidated_tape)

if not is_valid:
    print("[ERROR] Consolidated tape falló la validación")
    raise ValueError("Tape inválido")

# Mostrar estadísticas
print(f"\n[INFO] Estadísticas del Consolidated Tape:")
print(f"   - Total timestamps únicos: {len(consolidated_tape):,}")
print(f"   - Columnas totales: {len(consolidated_tape.columns)}")
print(f"   - Venues incluidos: {len(clean_data)}")

# Mostrar preview
print(f"\n[INFO] Preview del Consolidated Tape (primeras 5 filas):")
display_cols = ['epoch'] + [col for col in consolidated_tape.columns if '_bid' in col or '_ask' in col][:8]
print(consolidated_tape[display_cols].head())


2025-12-09 14:23:47,479 - consolidator_module - INFO - ConsolidatedTape initialized: time_bin=100ms



FASE 3: CONSOLIDATED TAPE

[INFO] Consolidando datos de todos los venues...

CREANDO CONSOLIDATED TAPE
  Venues a consolidar: ['AQEU', 'XMAD', 'CEUX', 'TQEX']
   AQEU: 306 snapshots preparados
   XMAD: 307 snapshots preparados
   CEUX: 306 snapshots preparados
   TQEX: 307 snapshots preparados

  Usando XMAD como base (307 rows)
    Merging con TQEX... OK (307 rows)
    Merging con AQEU... OK (307 rows)
    Merging con CEUX... OK (307 rows)

  [OK] Tape consolidado creado: (307, 17)
    - Timestamps únicos: 307
    - Columnas totales: 17

  Aplicando forward fill...
    NaNs antes: 0
    NaNs después: 0

  Tape final: (307, 17)

  [ESTADISTICAS DE COBERTURA]
    AQEU: 307 rows válidas (100.0% cobertura)
    XMAD: 307 rows válidas (100.0% cobertura)
    CEUX: 307 rows válidas (100.0% cobertura)
    TQEX: 307 rows válidas (100.0% cobertura)

[OK] Validando consolidated tape...

VALIDANDO CONSOLIDATED TAPE
  [OK] No NaNs después de las primeras 100 filas
  [OK] Timestamps monotónicamente

---

## FASE 4: DETECCIÓN DE SEÑALES {#fase4}

### Objetivo
Detectar oportunidades donde `MAX(bid) > MIN(ask)`.

### Algoritmo Paso a Paso

```
┌─────────────────────────────────────┐
│  PASO 1: Identificar columnas       │
│  • bid_cols = [XMAD_bid, AQXE_bid,  │
│                CEUX_bid, TRQX_bid]  │
│  • ask_cols = [XMAD_ask, AQXE_ask,  │
│                CEUX_ask, TRQX_ask]  │
└──────────────┬──────────────────────┘
               │
┌──────────────▼──────────────────────┐
│  PASO 2: Calcular Global Max Bid   │
│  ─────────────────────────────────  │
│  En cada momento:                    │
│  max_bid = MAX(todos los bids)      │
│  venue_max_bid = venue con max_bid  │
│                                      │
│  Ejemplo:                            │
│  XMAD: 10.52€                        │
│  AQXE: 10.49€                        │
│  CEUX: 10.48€                        │
│  TRQX: 10.50€                        │
│  → max_bid = 10.52€ (XMAD)          │
└──────────────┬──────────────────────┘
               │
┌──────────────▼──────────────────────┐
│  PASO 3: Calcular Global Min Ask    │
│  ─────────────────────────────────  │
│  En cada momento:                    │
│  min_ask = MIN(todos los asks)      │
│  venue_min_ask = venue con min_ask  │
│                                      │
│  Ejemplo:                            │
│  XMAD: 10.54€                        │
│  AQXE: 10.51€                        │
│  CEUX: 10.53€                        │
│  TRQX: 10.52€                        │
│  → min_ask = 10.51€ (AQXE)          │
└──────────────┬──────────────────────┘
               │
┌──────────────▼──────────────────────┐
│  PASO 4: Aplicar Regla de Oro       │
│  ─────────────────────────────────  │
│  signal = 1 si max_bid > min_ask    │
│  signal = 0 si no                   │
│                                      │
│  Ejemplo:                            │
│  max_bid = 10.52€ (XMAD)            │
│  min_ask = 10.51€ (AQXE)            │
│  10.52€ > 10.51€ → [OK] Oportunidad!  │
└──────────────┬──────────────────────┘
               │
┌──────────────▼──────────────────────┐
│  PASO 5: Calcular Cantidades        │
│  ─────────────────────────────────  │
│  • Extraer qty del venue_max_bid    │
│  • Extraer qty del venue_min_ask    │
│  • executable_qty = min(bid_qty,    │
│                        ask_qty)     │
└──────────────┬──────────────────────┘
               │
┌──────────────▼──────────────────────┐
│  PASO 6: Calcular Profit            │
│  ─────────────────────────────────  │
│  theoretical_profit = max_bid -     │
│                       min_ask        │
│  total_profit = theoretical_profit  │
│                  × executable_qty    │
└──────────────┬──────────────────────┘
               │
┌──────────────▼──────────────────────┐
│  PASO 7: Rising Edge Detection      │
│  ─────────────────────────────────  │
│  Solo contar la PRIMERA aparición   │
│  de cada oportunidad continua       │
│                                      │
│  Si una oportunidad persiste 1000   │
│  snapshots, solo la contamos UNA vez│
└─────────────────────────────────────┘
```

### Rising Edge Detection

**Problema:** Si una oportunidad persiste durante muchos snapshots, no queremos contarla múltiples veces.

**Solución:** Rising Edge Detection identifica solo la **primera aparición** de cada oportunidad continua.

```
Snapshot 1: signal = 0 → No oportunidad
Snapshot 2: signal = 1 → RISING EDGE (nueva oportunidad)
Snapshot 3: signal = 1 → No (continuación de oportunidad anterior)
Snapshot 4: signal = 1 → No (continuación)
Snapshot 5: signal = 0 → Oportunidad desapareció
Snapshot 6: signal = 1 → RISING EDGE (nueva oportunidad)
```

**Algoritmo:**
```python
prev_signal = signal.shift(1, fill_value=0)
is_rising_edge = (signal == 1) & (prev_signal == 0)
```


In [23]:
# ============================================================================
# FASE 4: DETECCIÓN DE SEÑALES DE ARBITRAJE
# ============================================================================

print("\n" + "=" * 80)
print("FASE 4: DETECCIÓN DE SEÑALES DE ARBITRAJE")
print("=" * 80)

# Inicializar generador de señales
signal_gen = SignalGenerator()

# Inicializar lista de trades ejecutados (vacía al inicio)
executed_trades = []

# Detectar oportunidades
print("\n[INFO] Detectando oportunidades de arbitraje...")
signals_df = signal_gen.detect_opportunities(
    consolidated_tape,
    executed_trades=executed_trades,
    isin=test_isin
)

if signals_df is None or len(signals_df) == 0:
    print("[ERROR] No se detectaron señales")
    signals_df = pd.DataFrame()
else:
    # Mostrar resumen
    rising_edges = signals_df[signals_df['is_rising_edge']]
    total_opps = len(rising_edges)
    
    print(f"\n[OK] Señales detectadas:")
    print(f"   - Total snapshots analizados: {len(signals_df):,}")
    print(f"   - Snapshots con arbitraje: {signals_df['signal'].sum():,}")
    print(f"   - Rising edges (oportunidades únicas): {total_opps:,}")
    
    if total_opps > 0:
        total_profit = rising_edges['total_profit'].sum()
        avg_profit = rising_edges['total_profit'].mean()
        print(f"   - Profit teórico total: €{total_profit:,.2f}")
        print(f"   - Profit medio por oportunidad: €{avg_profit:.2f}")
        
        # Mostrar primeras oportunidades
        print(f"\n[INFO] Primeras 5 oportunidades detectadas:")
        opp_cols = ['epoch', 'venue_max_bid', 'venue_min_ask', 
                   'executable_qty', 'theoretical_profit', 'total_profit']
        available_cols = [c for c in opp_cols if c in rising_edges.columns]
        print(rising_edges[available_cols].head().to_string(index=False))
    
    # Analizar pares de venues
    print("\n[INFO] Analizando pares de venues...")
    venue_pairs = signal_gen.analyze_venue_pairs(signals_df)
    if venue_pairs is not None and len(venue_pairs) > 0:
        print("\n[INFO] Top pares de venues por profit:")
        print(venue_pairs.head().to_string(index=False))


2025-12-09 14:23:47,801 - signal_generator_module - INFO - No se encontraron oportunidades de arbitraje



FASE 4: DETECCIÓN DE SEÑALES DE ARBITRAJE

[INFO] Detectando oportunidades de arbitraje...
[ERROR] No se detectaron señales


---

## FASE 5: EJECUCIÓN INSTANTÁNEA {#fase5}

### Objetivo
Calcular el profit real asumiendo ejecución instantánea (latencia = 0, sin comisiones).

### Asunciones del Modelo

```
┌─────────────────────────────────────┐
│  Latencia = 0                        │
│  ─────────────────────────────────  │
│  • Detección y ejecución simultáneas │
│  • No hay delay entre detectar y     │
│    ejecutar                          │
│  • execution_epoch = signal_epoch    │
└─────────────────────────────────────┘

┌─────────────────────────────────────┐
│  Sin Comisiones                      │
│  ─────────────────────────────────  │
│  • No hay costos de trading          │
│  • No hay fees de exchange           │
│  • Profit teórico = Profit real      │
└─────────────────────────────────────┘
```

### Cálculo de Profit Real

```
real_profit = theoretical_profit
real_total_profit = total_profit
profit_category = 'Profitable' (todas)
```

**Nota:** En un modelo más realista, habría que:
1. Simular latencia (time machine)
2. Recalcular precios en el momento de ejecución
3. Aplicar comisiones
4. Verificar que la oportunidad sigue existiendo

Pero en este modelo simplificado, asumimos ejecución perfecta e instantánea.


In [24]:
# ============================================================================
# FASE 5: EJECUCIÓN INSTANTÁNEA (LATENCIA = 0, SIN COMISIONES)
# ============================================================================

print("\n" + "=" * 80)
print("FASE 5: EJECUCIÓN INSTANTÁNEA")
print("=" * 80)

print("\n[INFO] Asunciones del modelo:")
print("   • Latencia = 0 (ejecución instantánea)")
print("   • Sin comisiones de mercado")
print("   • Profit teórico = Profit real")

exec_df = None

if signals_df is not None and len(signals_df) > 0:
    # Filtrar solo rising edges (oportunidades únicas)
    rising_edges = signals_df[signals_df['is_rising_edge']]
    
    if len(rising_edges) > 0:
        # Crear DataFrame de ejecuciones
        exec_df = rising_edges.copy()
        exec_df['execution_epoch'] = exec_df['epoch']  # Ejecución instantánea
        exec_df['executed_qty'] = exec_df['executable_qty']
        exec_df['real_profit'] = exec_df['theoretical_profit']  # Sin pérdida por latencia
        exec_df['real_total_profit'] = exec_df['total_profit']  # Profit total real
        exec_df['profit_category'] = 'Profitable'  # Todas son profitable
        
        print(f"\n[OK] Ejecuciones simuladas:")
        print(f"   - Total oportunidades ejecutables: {len(exec_df):,}")
        print(f"   - Profit total real: €{exec_df['real_total_profit'].sum():,.2f}")
        
        # Mostrar primeras ejecuciones
        print(f"\n[INFO] Primeras 5 ejecuciones:")
        exec_cols = ['epoch', 'execution_epoch', 'venue_max_bid', 'venue_min_ask',
                    'executed_qty', 'real_profit', 'real_total_profit']
        available_cols = [c for c in exec_cols if c in exec_df.columns]
        print(exec_df[available_cols].head().to_string(index=False))
    else:
        print("[WARNING] No hay rising edges para ejecutar")
else:
    print("[WARNING] No hay señales para ejecutar")



FASE 5: EJECUCIÓN INSTANTÁNEA

[INFO] Asunciones del modelo:
   • Latencia = 0 (ejecución instantánea)
   • Sin comisiones de mercado
   • Profit teórico = Profit real
[WARNING] No hay señales para ejecutar


---

## FASE 6: ANÁLISIS FINAL {#fase6}

### Objetivo
Generar métricas agregadas y estadísticas descriptivas.

### Métricas Calculadas

```
┌─────────────────────────────────────┐
│  Métricas por Oportunidad           │
│  ─────────────────────────────────  │
│  • Total oportunidades detectadas   │
│  • Total profit teórico              │
│  • Profit medio por oportunidad     │
│  • Profit máximo                     │
│  • Cantidad media ejecutable         │
└─────────────────────────────────────┘

┌─────────────────────────────────────┐
│  Métricas de ROI                    │
│  ─────────────────────────────────  │
│  • Capital requerido                │
│  • Profit total                     │
│  • ROI porcentual                   │
│  • Profit por unidad de capital     │
└─────────────────────────────────────┘
```

### Análisis de Pares de Venues

Identifica qué combinaciones de venues generan más oportunidades:

```
Ejemplo:
Buy@AQXE / Sell@XMAD: 150 oportunidades, €450.00 total profit
Buy@CEUX / Sell@XMAD: 89 oportunidades, €234.50 total profit
...
```


In [25]:
# ============================================================================
# FASE 6: ANÁLISIS FINAL
# ============================================================================

print("\n" + "=" * 80)
print("FASE 6: ANÁLISIS FINAL")
print("=" * 80)

metrics = None
roi_metrics = None

if signals_df is not None and exec_df is not None and len(exec_df) > 0:
    analyzer = ArbitrageAnalyzer()
    
    print("\n[INFO] Analizando oportunidades...")
    metrics = analyzer.analyze_opportunities(signals_df, exec_df)
    
    print("\n[INFO] Estimando ROI...")
    # Sin comisiones: trading_costs_bps = 0
    roi_metrics = analyzer.estimate_roi(metrics, trading_costs_bps=0.0, capital_eur=100000)
    
    # Mostrar métricas
    if metrics:
        print("\n[OK] Métricas del análisis:")
        metrics_df = pd.DataFrame([metrics])
        print(metrics_df.to_string(index=False))
    
    if roi_metrics:
        print("\n[OK] Métricas de ROI:")
        roi_df = pd.DataFrame([roi_metrics])
        print(roi_df.to_string(index=False))
    
    # Generar reporte
    print("\n[INFO] Generando reporte final...")
    analyzer.generate_summary_report(
        metrics,
        roi_metrics,
        output_path=config.OUTPUT_DIR / f'report_{test_isin}.txt'
    )
    print("[OK] Reporte generado")
else:
    print("[WARNING] No hay datos suficientes para análisis final")



FASE 6: ANÁLISIS FINAL
[WARNING] No hay datos suficientes para análisis final


---

## VISUALIZACIONES Y REPORTES {#visualizaciones}

### Gráficas Generadas

1. **Consolidated Tape**: Muestra la evolución de precios bid/ask de todos los venues
2. **Señales Detectadas**: Visualiza las oportunidades de arbitraje en el tiempo
3. **Análisis de Pares**: Gráfico de barras con profit por par de venues

### Archivos Generados

- `opportunities_{ISIN}.csv`: Todas las oportunidades detectadas
- `execution_{ISIN}.csv`: Todas las ejecuciones simuladas
- `report_{ISIN}.txt`: Reporte de métricas y ROI
- `complete_report_{ISIN}.md`: Documento Markdown completo
- `figures/`: Directorio con todas las gráficas


In [26]:
# ============================================================================
# VISUALIZACIONES Y EXPORTACIÓN
# ============================================================================

print("\n" + "=" * 80)
print("VISUALIZACIONES Y EXPORTACIÓN")
print("=" * 80)

if signals_df is not None and len(signals_df) > 0:
    # Visualizar señales
    print("\n[INFO] Generando visualizaciones...")
    signal_gen.visualize_signals(signals_df, test_isin)
    
    # Visualizar consolidated tape
    tape_builder.visualize_tape(consolidated_tape, test_isin)
    
    # Exportar oportunidades
    print("\n[INFO] Exportando oportunidades...")
    signal_gen.export_opportunities(
        signals_df,
        output_path=config.OUTPUT_DIR / f"opportunities_{test_isin}.csv"
    )
    
    # Exportar ejecuciones
    if exec_df is not None and len(exec_df) > 0:
        exec_cols = ['epoch', 'execution_epoch', 'venue_max_bid', 'venue_min_ask',
                    'executed_qty', 'real_profit', 'real_total_profit', 'profit_category']
        exec_df[[c for c in exec_cols if c in exec_df.columns]].to_csv(
            config.OUTPUT_DIR / f"execution_{test_isin}.csv",
            index=False
        )
        print("[OK] Ejecuciones exportadas")
    
    print("\n[OK] Visualizaciones generadas")
    print(f"   [INFO] Archivos guardados en: {config.OUTPUT_DIR}")
    print(f"   [INFO] Gráficas guardadas en: {config.FIGURES_DIR}")
else:
    print("[WARNING] No hay señales para visualizar")



VISUALIZACIONES Y EXPORTACIÓN
[WARNING] No hay señales para visualizar


---

## RESUMEN FINAL

### Resultados del Análisis

A continuación se muestra un resumen completo de todos los resultados obtenidos.


In [27]:
# ============================================================================
# RESUMEN FINAL
# ============================================================================

print("\n" + "=" * 80)
print("RESUMEN DEL ANÁLISIS")
print("=" * 80)

print(f"\n[INFO] ISIN: {test_isin}")
print(f"   - Dataset usado: {DATA_DIR_NAME}")
print(f"   - Total snapshots en tape: {len(consolidated_tape):,}")
print(f"   - Venues incluidos: {len(clean_data)}")

if signals_df is not None and len(signals_df) > 0:
    total_opportunities = signals_df['is_rising_edge'].sum()
    total_profit = signals_df[signals_df['is_rising_edge']]['total_profit'].sum()
    
    print(f"\n[INFO] Oportunidades:")
    print(f"   - Detectadas: {total_opportunities:,}")
    print(f"   - Profit total (ejecución instantánea): €{total_profit:,.2f}")
    
    if total_opportunities > 0:
        avg_profit = signals_df[signals_df['is_rising_edge']]['total_profit'].mean()
        print(f"   - Profit medio por oportunidad: €{avg_profit:.2f}")

if exec_df is not None and len(exec_df) > 0:
    profitable_ops = len(exec_df)
    real_profit = exec_df['real_total_profit'].sum()
    print(f"\n[INFO] Ejecuciones:")
    print(f"   - Oportunidades ejecutadas: {profitable_ops:,}")
    print(f"   - Profit real total: €{real_profit:,.2f}")

if roi_metrics:
    print(f"\n[INFO] ROI:")
    print(f"   - ROI estimado: {roi_metrics.get('roi_pct', 0):.4f}%")

print("\n" + "=" * 80)
print("[ÉXITO] ANÁLISIS COMPLETADO CON ÉXITO")
print("=" * 80)

# Mostrar archivos generados
print(f"\n[INFO] Archivos generados:")
print(f"   - Log: {config.OUTPUT_DIR / 'arbitrage_system.log'}")
if signals_df is not None and len(signals_df) > 0:
    print(f"   - Oportunidades: {config.OUTPUT_DIR / f'opportunities_{test_isin}.csv'}")
if exec_df is not None and len(exec_df) > 0:
    print(f"   - Ejecuciones: {config.OUTPUT_DIR / f'execution_{test_isin}.csv'}")
if metrics:
    print(f"   - Reporte: {config.OUTPUT_DIR / f'report_{test_isin}.txt'}")
print(f"   - Figuras: {config.FIGURES_DIR}")



RESUMEN DEL ANÁLISIS

[INFO] ISIN: ES0113900J37
   - Dataset usado: DATA_SMALL
   - Total snapshots en tape: 307
   - Venues incluidos: 4

[ÉXITO] ANÁLISIS COMPLETADO CON ÉXITO

[INFO] Archivos generados:
   - Log: c:\Users\Pc\Downloads\TAREA_RENTA_VARIABLE\output\arbitrage_system.log
   - Figuras: c:\Users\Pc\Downloads\TAREA_RENTA_VARIABLE\output\figures


---

## EXPLORACIÓN ADICIONAL

### Visualizar Datos Intermedios

Puedes explorar los datos en cualquier punto del pipeline usando las variables:

- `raw_data`: Datos después de la carga
- `clean_data`: Datos después de la limpieza
- `consolidated_tape`: Tape consolidado con todos los venues
- `signals_df`: DataFrame con todas las señales detectadas
- `exec_df`: DataFrame con todas las ejecuciones simuladas

### Ejemplos de Exploración

```python
# Ver distribución de precios en el consolidated tape
consolidated_tape[['XMAD_bid', 'XMAD_ask', 'AQXE_bid', 'AQXE_ask']].describe()

# Ver oportunidades por venue pair
signals_df[signals_df['is_rising_edge']].groupby(['venue_max_bid', 'venue_min_ask']).size()

# Ver evolución temporal del profit
signals_df[signals_df['is_rising_edge']].plot(x='epoch', y='total_profit', kind='line')
```


In [28]:
# ============================================================================
# EXPLORACIÓN ADICIONAL - Celdas opcionales para análisis personalizado
# ============================================================================

# Descomenta las siguientes líneas para explorar los datos:

# 1. Ver estadísticas del consolidated tape
# print("\n[INFO] Estadísticas de precios en Consolidated Tape:")
# price_cols = [col for col in consolidated_tape.columns if '_bid' in col or '_ask' in col]
# print(consolidated_tape[price_cols].describe())

# 2. Ver distribución de oportunidades por venue pair
# if signals_df is not None and len(signals_df) > 0:
#     rising_edges = signals_df[signals_df['is_rising_edge']]
#     print("\n[INFO] Oportunidades por par de venues:")
#     venue_pairs = rising_edges.groupby(['venue_max_bid', 'venue_min_ask']).agg({
#         'total_profit': ['count', 'sum', 'mean']
#     })
#     print(venue_pairs)

# 3. Ver evolución temporal del profit
# if signals_df is not None and len(signals_df) > 0:
#     rising_edges = signals_df[signals_df['is_rising_edge']]
#     plt.figure(figsize=(12, 6))
#     plt.plot(rising_edges['epoch'], rising_edges['total_profit'], marker='o', markersize=2)
#     plt.xlabel('Epoch (nanosegundos)')
#     plt.ylabel('Profit (€)')
#     plt.title('Evolución Temporal del Profit')
#     plt.grid(True)
#     plt.show()

print("\n[INFO] Usa las celdas anteriores para explorar los datos en detalle")



[INFO] Usa las celdas anteriores para explorar los datos en detalle


---

## ANÁLISIS COMPLETO DE TODOS LOS ISINs (Igual que el otro código)

Esta sección procesa TODOS los ISINs disponibles y genera:
1. **Money Table**: Tabla de profit por ISIN y latencia
2. **Decay Chart**: Gráfico de decay de profit con latencia
3. **Top Opportunities**: Top 5 ISINs más rentables
4. **Summary**: Respuestas a las preguntas clave

**Flujo igual al otro código:**
- Descubrir todos los ISINs
- Para cada ISIN: cargar → consolidar → detectar → rising edge → simular latencias
- Generar tablas y gráficos finales


In [29]:
# ============================================================================
# ANÁLISIS COMPLETO DE TODOS LOS ISINs
# ============================================================================
# Igual que el otro código: procesar todos los ISINs y generar Money Table

from collections import defaultdict
from latency_simulator_module import simulate_latency_with_losses

# Configuración (igual que el otro código)
DATE = "2025-11-07"  # Ajustar según tus datos
LATENCY_LEVELS = config.LATENCY_BUCKETS  # [0, 100, 500, 1000, ...]

print("=" * 80)
print("ANÁLISIS COMPLETO DE TODOS LOS ISINs")
print("=" * 80)

# Descubrir todos los ISINs
print("\n[INFO] Descubriendo ISINs...")
all_isins = loader.discover_isins()
print(f"Found {len(all_isins)} unique ISINs\n")

# Storage for results (igual que el otro código)
money_table_data = []

# Process each ISIN (igual que el otro código)
for isin_idx, isin in enumerate(all_isins, 1):
    print(f"[{isin_idx}/{len(all_isins)}] Processing ISIN: {isin}")
    
    # Load data
    data_dict_raw = loader.load_isin_data(isin)
    
    if not data_dict_raw:
        print(f"  No data found for {isin}")
        continue
    
    # Convertir formato: nuestro código usa {'mic': {'qte': df, 'sts': df}}
    # El otro código usa {'mic': (qte_df, sts_df)}
    data_dict = {}
    for mic, venue_data in data_dict_raw.items():
        qte_df = venue_data.get('qte', pd.DataFrame())
        sts_df = venue_data.get('sts', pd.DataFrame())
        if not qte_df.empty:
            data_dict[mic] = (qte_df, sts_df)
    
    if not data_dict:
        print(f"  No valid data found for {isin}")
        continue
    
    print(f"  Found data from {len(data_dict)} exchange(s): {list(data_dict.keys())}")
    
    # Clean data (aplicar limpieza)
    cleaner = DataCleaner()
    clean_data = cleaner.clean_all_venues(data_dict_raw)
    
    if not clean_data:
        print(f"  No data after cleaning for {isin}")
        continue
    
    # Create consolidated tape
    tape_builder = ConsolidatedTape(time_bin_ms=100)
    consolidated = tape_builder.create_tape(clean_data)
    
    if consolidated is None or consolidated.empty:
        print(f"  No consolidated tape created for {isin}")
        continue
    
    # Detect arbitrage opportunities (igual que el otro código)
    signal_gen = SignalGenerator()
    opportunities = signal_gen.detect_arbitrage_opportunities(consolidated)
    
    if opportunities.empty:
        print(f"  No arbitrage opportunities found for {isin}")
        continue
    
    # Apply rising edge detection (igual que el otro código)
    opportunities = signal_gen.apply_rising_edge(opportunities)
    
    # Filtrar solo las que pasaron el rising edge
    opportunities = opportunities[opportunities.get('is_rising_edge', False)].copy()
    
    if opportunities.empty:
        print(f"  No opportunities after rising edge for {isin}")
        continue
    
    print(f"  Found {len(opportunities)} arbitrage opportunities (after rising edge)")
    
    # Simulate latency for each level (igual que el otro código)
    profits_by_latency = {}
    for latency_us in LATENCY_LEVELS:
        profit = simulate_latency_with_losses(opportunities, consolidated, latency_us)
        profits_by_latency[latency_us] = profit
        
        money_table_data.append({
            'ISIN': isin,
            'Latency_us': latency_us,
            'Profit_EUR': profit
        })
    
    # Print profits for all latencies for this ISIN (igual que el otro código)
    print(f"\n  Profits by Latency for {isin}:")
    print(f"  {'Latency (µs)':<15} {'Latency (ms)':<15} {'Profit/Loss (€)':<20} {'% of 0 latency':<15}")
    print(f"  {'-'*65}")
    
    zero_latency_profit = profits_by_latency[0]
    for latency_us in LATENCY_LEVELS:
        profit = profits_by_latency[latency_us]
        latency_ms = latency_us / 1000
        if zero_latency_profit != 0:
            pct = (profit / zero_latency_profit * 100) if zero_latency_profit != 0 else 0
            pct_str = f"{pct:.1f}%"
        else:
            pct_str = "N/A"
        
        # Color coding: negative = loss, positive = profit
        profit_str = f"€{profit:,.2f}"
        if profit < 0:
            profit_str = f"€{profit:,.2f} (LOSS)"
        
        print(f"  {latency_us:<15} {latency_ms:<15.3f} {profit_str:<20} {pct_str:<15}")
    
    print(f"  {'-'*65}")
    print(f"  Total profit at 0 latency: €{zero_latency_profit:,.2f}\n")

print("\n" + "="*80)
print("ANALYSIS COMPLETE")
print("="*80)


ANÁLISIS COMPLETO DE TODOS LOS ISINs

[INFO] Descubriendo ISINs...

DESCUBRIENDO ISINs DISPONIBLES
 Encontrados 1 ISINs unicos
  Primeros 5: ['ES0113900J37']
Found 1 unique ISINs

[1/1] Processing ISIN: ES0113900J37

CARGANDO DATOS PARA ISIN: ES0113900J37
  Archivos QTE encontrados: 4

  [PROCESANDO] Venue: AQEU


2025-12-09 14:23:49,533 - data_loader_module - INFO -   [OK] QTE_2025-11-07_ES0113900J37_SANe_AQEU_1.csv.gz: 101,610 filas válidas (de 101,616)
2025-12-09 14:23:49,544 - data_loader_module - INFO -   [OK] STS_2025-11-07_ES0113900J37_SANe_AQEU_1.csv.gz: 6 filas
2025-12-09 14:23:49,587 - data_loader_module - INFO -   [DEBUG] AQEU: Columnas disponibles en QTE: ['session', 'inst_id', 'sequence', 'isin', 'ticker', 'mic', 'currency', 'epoch', 'event_timestamp', 'bloombergTicker', 'ord_bid_0', 'qty_bid_0', 'px_bid_0', 'px_ask_0', 'qty_ask_0', 'ord_ask_0', 'ord_bid_1', 'qty_bid_1', 'px_bid_1', 'px_ask_1']
2025-12-09 14:23:49,590 - data_loader_module - INFO -   [DEBUG] AQEU: Total columnas: 70
2025-12-09 14:23:49,590 - data_loader_module - INFO -   [DEBUG] AQEU: Filas en QTE: 101610


    [OK] AQEU: 101,610 snapshots

  [PROCESANDO] Venue: XMAD


2025-12-09 14:23:53,594 - data_loader_module - INFO -   [OK] QTE_2025-11-07_ES0113900J37_SAN_XMAD_1.csv.gz: 364,903 filas válidas (de 364,912)
2025-12-09 14:23:53,614 - data_loader_module - INFO -   [OK] STS_2025-11-07_ES0113900J37_SAN_XMAD_1.csv.gz: 7 filas
2025-12-09 14:23:53,788 - data_loader_module - INFO -   [DEBUG] XMAD: Columnas disponibles en QTE: ['session', 'inst_id', 'sequence', 'isin', 'ticker', 'mic', 'currency', 'epoch', 'event_timestamp', 'bloombergTicker', 'ord_bid_0', 'qty_bid_0', 'px_bid_0', 'px_ask_0', 'qty_ask_0', 'ord_ask_0', 'ord_bid_1', 'qty_bid_1', 'px_bid_1', 'px_ask_1']
2025-12-09 14:23:53,789 - data_loader_module - INFO -   [DEBUG] XMAD: Total columnas: 70
2025-12-09 14:23:53,790 - data_loader_module - INFO -   [DEBUG] XMAD: Filas en QTE: 364903


    [OK] XMAD: 364,903 snapshots

  [PROCESANDO] Venue: CEUX


...

    [OK] Snapshots finales: 101,559 (99.95% retenido)

  [LIMPIEZA] XMAD...
    Snapshots iniciales: 364,903


2025-12-09 14:23:57,643 - data_cleaner_module - INFO -     Removed 1 magic numbers (0.00%)
2025-12-09 14:23:57,646 - data_cleaner_module - INFO -     Códigos esperados para XMAD: [5832713, 5832756]
2025-12-09 14:23:57,647 - data_cleaner_module - INFO -     Códigos encontrados en STS: [np.int64(5832754), np.int64(5832755), np.int64(5832756), np.int64(5832757), np.int64(5832758), np.int64(5832762), np.int64(5832763)]
2025-12-09 14:23:57,648 - data_cleaner_module - INFO -     Códigos que coinciden: [5832756]
2025-12-09 14:23:57,808 - data_cleaner_module - INFO -     Snapshots con estado asignado: 364,901 (100.00%)
2025-12-09 14:23:57,809 - data_cleaner_module - INFO -     Snapshots sin estado asignado: 1 (0.00%)
2025-12-09 14:23:58,613 - data_cleaner_module - INFO -     Distribución de estados encontrados:
2025-12-09 14:23:58,616 - data_cleaner_module - INFO -       5832756: 362,896 snapshots ([VALID])
2025-12-09 14:23:58,617 - data_cleaner_module - INFO -       5832757: 1,393 snapshots (

    [OK] Snapshots finales: 362,894 (99.45% retenido)

  [LIMPIEZA] CEUX...
    Snapshots iniciales: 78,462


2025-12-09 14:23:59,991 - data_cleaner_module - INFO -       12255233: 78,375 snapshots ([VALID])
2025-12-09 14:23:59,992 - data_cleaner_module - INFO -       12255237: 50 snapshots ([INVALID])
2025-12-09 14:23:59,992 - data_cleaner_module - INFO -       12255244: 37 snapshots ([INVALID])
2025-12-09 14:24:00,016 - data_cleaner_module - INFO -     [OK] Removed 87 non-trading snapshots (0.11%)
2025-12-09 14:24:00,018 - data_cleaner_module - INFO -     [OK] Kept 78,375 continuous trading snapshots (99.89%)
2025-12-09 14:24:00,161 - data_cleaner_module - INFO -     Códigos esperados para TQEX: [7608181]
2025-12-09 14:24:00,162 - data_cleaner_module - INFO -     Códigos encontrados en STS: [np.int64(7608181), np.int64(7608182), np.int64(7608183)]
2025-12-09 14:24:00,162 - data_cleaner_module - INFO -     Códigos que coinciden: [7608181]
2025-12-09 14:24:00,179 - data_cleaner_module - INFO -     Snapshots con estado asignado: 43,316 (100.00%)
2025-12-09 14:24:00,180 - data_cleaner_module - I

    [OK] Snapshots finales: 78,375 (99.89% retenido)

  [LIMPIEZA] TQEX...
    Snapshots iniciales: 43,316
    [OK] Snapshots finales: 43,316 (100.00% retenido)

  [MÉTRICAS DE CALIDAD AGREGADAS]
    Filas originales: 588,291
    Eliminadas por magic numbers: 1 (0.00%)
    Eliminadas por status inválido: 2,144 (0.36%)
    Eliminadas por validaciones: 2 (0.00%)
    Filas finales: 586,144 (99.64% retenido)

[EXITO] Limpieza completada para 4 venues

CREANDO CONSOLIDATED TAPE
  Venues a consolidar: ['AQEU', 'XMAD', 'CEUX', 'TQEX']
   AQEU: 306 snapshots preparados
   XMAD: 307 snapshots preparados
   CEUX: 306 snapshots preparados
   TQEX: 307 snapshots preparados

  Usando XMAD como base (307 rows)
    Merging con TQEX... OK (307 rows)
    Merging con AQEU... OK (307 rows)
    Merging con CEUX... OK (307 rows)

  [OK] Tape consolidado creado: (307, 17)
    - Timestamps únicos: 307
    - Columnas totales: 17

  Aplicando forward fill...
    NaNs antes: 0
    NaNs después: 0

  Tape final:

## Deliverable 1: The "Money Table"

Tabla pivot con profit por ISIN y latencia (igual que el otro código)


In [30]:
# ============================================================================
# DELIVERABLE 1: THE "MONEY TABLE"
# ============================================================================
# Igual que el otro código

if money_table_data:
    money_df = pd.DataFrame(money_table_data)
    
    # Create pivot table
    pivot = money_df.pivot_table(
        index='ISIN',
        columns='Latency_us',
        values='Profit_EUR',
        aggfunc='sum'
    )
    
    # Add TOTAL row
    pivot.loc['TOTAL'] = pivot.sum()
    
    print("MONEY TABLE: Total Realized Profit/Loss by ISIN and Latency")
    print("="*120)
    print(pivot.to_string())
    
    # Summary by latency
    print("\n" + "="*80)
    print("SUMMARY BY LATENCY (All ISINs Combined)")
    print("="*80)
    
    # Calculate profit at 0 latency for percentage calculation
    profit_at_zero = pivot.loc['TOTAL', 0] if 0 in pivot.columns else 0
    
    summary_df = pd.DataFrame({
        'Latency (µs)': LATENCY_LEVELS,
        'Total Profit/Loss (€)': [pivot.loc['TOTAL', lat] for lat in LATENCY_LEVELS],
        'Latency (ms)': [lat / 1000 for lat in LATENCY_LEVELS],
        '% of 0 latency': [
            (pivot.loc['TOTAL', lat] / profit_at_zero * 100) if profit_at_zero != 0 else 0 
            for lat in LATENCY_LEVELS
        ]
    })
    
    print(summary_df.to_string(index=False))
    
    # Count arbitrage opportunities by exchange direction (Buy -> Sell)
    print("\n" + "="*80)
    print("ARBITRAGE OPPORTUNITIES BY EXCHANGE DIRECTION")
    print("="*80)
    
    # Get ISINs with opportunities at 0 latency
    isins_with_opps = money_df[(money_df['Latency_us'] == 0) & (money_df['Profit_EUR'] > 0)]['ISIN'].unique()
    
    # Count opportunities by exchange direction (Buy Exchange -> Sell Exchange)
    exchange_direction_counts = defaultdict(int)
    
    for isin in isins_with_opps:
        # Load data for this ISIN
        data_dict_raw = loader.load_isin_data(isin)
        
        if not data_dict_raw:
            continue
        
        # Clean and consolidate
        cleaner = DataCleaner()
        clean_data = cleaner.clean_all_venues(data_dict_raw)
        
        if not clean_data:
            continue
        
        tape_builder = ConsolidatedTape(time_bin_ms=100)
        consolidated = tape_builder.create_tape(clean_data)
        
        if consolidated.empty:
            continue
        
        # Detect arbitrage opportunities
        signal_gen = SignalGenerator()
        opportunities = signal_gen.detect_arbitrage_opportunities(consolidated)
        
        if opportunities.empty:
            continue
        
        # Apply rising edge detection
        opportunities = signal_gen.apply_rising_edge(opportunities)
        opportunities = opportunities[opportunities.get('is_rising_edge', False)].copy()
        
        if opportunities.empty:
            continue
        
        # Count by exchange direction (Buy -> Sell)
        for _, opp in opportunities.iterrows():
            buy_ex = opp.get('buy_exchange', '')
            sell_ex = opp.get('sell_exchange', '')
            
            if buy_ex and sell_ex:
                # Count direction separately (Buy -> Sell)
                direction = f"{buy_ex} → {sell_ex}"
                exchange_direction_counts[direction] += 1
    
    # Sort by count (descending)
    sorted_directions = sorted(exchange_direction_counts.items(), key=lambda x: x[1], reverse=True)
    
    if sorted_directions:
        print(f"\n{'Rank':<8} {'Exchange Direction (Buy → Sell)':<40} {'Opportunities':<15}")
        print(f"{'-'*8} {'-'*40} {'-'*15}")
        
        for rank, (direction, count) in enumerate(sorted_directions, 1):
            print(f"{rank:<8} {direction:<40} {count:<15}")
        
        print(f"\nTotal unique exchange directions: {len(sorted_directions)}")
        print(f"Total opportunities: {sum(exchange_direction_counts.values())}")
    else:
        print("No exchange direction data available.")
    
else:
    print("No data available for Money Table.")


No data available for Money Table.


## Deliverable 2: The Decay Chart

Gráfico de decay de profit con latencia (igual que el otro código)


In [31]:
# ============================================================================
# DELIVERABLE 2: THE DECAY CHART
# ============================================================================
# Igual que el otro código

if money_table_data:
    money_df = pd.DataFrame(money_table_data)
    
    # Calculate totals by latency
    totals_by_latency = money_df.groupby('Latency_us')['Profit_EUR'].sum()
    
    latencies_ms = [lat / 1000 for lat in LATENCY_LEVELS]
    profits = [totals_by_latency.get(lat, 0) for lat in LATENCY_LEVELS]
    
    max_profit = profits[0]
    
    # Check if there are any losses
    has_losses = any(p < 0 for p in profits)
    
    # Calculate percentages
    percentages = [(p / max_profit * 100) if max_profit != 0 else 0 for p in profits]
    
    # Create figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot 1: Log scale (if all profits are positive)
    if not has_losses and all(p > 0 for p in profits):
        ax1.semilogy(latencies_ms, profits, 'b-o', linewidth=2, markersize=8)
        ax1.set_ylabel('Profit (€, log scale)', fontsize=12)
        ax1.set_title('Profit Decay with Latency (Log Scale)', fontsize=14, fontweight='bold')
    else:
        ax1.plot(latencies_ms, profits, 'b-o', linewidth=2, markersize=8)
        ax1.axhline(y=0, color='r', linestyle='--', linewidth=2, label='Break-even')
        ax1.set_ylabel('Profit/Loss (€)', fontsize=12)
        ax1.set_title('Profit Decay with Latency (Linear Scale)', fontsize=14, fontweight='bold')
        ax1.legend()
    
    ax1.set_xlabel('Latency (ms)', fontsize=12)
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Percentage of 0 latency profit
    ax2.plot(latencies_ms, percentages, 'g-o', linewidth=2, markersize=8)
    ax2.axhline(y=0, color='r', linestyle='--', linewidth=2, label='Break-even')
    ax2.axhline(y=100, color='b', linestyle=':', linewidth=1, alpha=0.5, label='100% (0 latency)')
    ax2.set_xlabel('Latency (ms)', fontsize=12)
    ax2.set_ylabel('% of Profit at 0 Latency', fontsize=12)
    ax2.set_title('Profit Decay as % of 0 Latency', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    
    plt.tight_layout()
    plt.savefig(config.FIGURES_DIR / 'decay_chart.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    # Print decay analysis
    print("\nDecay Analysis:")
    print(f"  Maximum profit (0 latency): €{max_profit:,.2f}")
    
    # Find profits at key latencies
    key_latencies = [1000, 10000, 100000]  # 1ms, 10ms, 100ms
    for lat_us in key_latencies:
        if lat_us in totals_by_latency.index:
            profit_at_latency = totals_by_latency[lat_us]
            print(f"  Profit/Loss at {lat_us/1000}ms: €{profit_at_latency:,.2f}")
else:
    print("No data available for Decay Chart.")


No data available for Decay Chart.


## Deliverable 3: Top Opportunities

Top 5 ISINs más rentables con detalles (igual que el otro código)


In [32]:
# ============================================================================
# DELIVERABLE 3: TOP OPPORTUNITIES
# ============================================================================
# Igual que el otro código

if money_table_data:
    money_df = pd.DataFrame(money_table_data)
    
    # Get top 5 ISINs by profit at 0 latency
    zero_latency = money_df[money_df['Latency_us'] == 0]
    top_isins = zero_latency.nlargest(5, 'Profit_EUR')
    
    print("TOP 5 MOST PROFITABLE ISINs (at 0 latency)")
    print("="*80)
    print()
    
    for rank, (_, row) in enumerate(top_isins.iterrows(), 1):
        isin = row['ISIN']
        total_profit = row['Profit_EUR']
        
        # Reload data to get detailed info
        data_dict_raw = loader.load_isin_data(isin)
        cleaner = DataCleaner()
        clean_data = cleaner.clean_all_venues(data_dict_raw)
        tape_builder = ConsolidatedTape(time_bin_ms=100)
        consolidated = tape_builder.create_tape(clean_data)
        signal_gen = SignalGenerator()
        opportunities = signal_gen.detect_arbitrage_opportunities(consolidated)
        opportunities = signal_gen.apply_rising_edge(opportunities)
        opportunities = opportunities[opportunities.get('is_rising_edge', False)].copy()
        
        if opportunities.empty:
            continue
        
        num_opps = len(opportunities)
        avg_profit = opportunities['total_profit'].mean()
        max_profit = opportunities['total_profit'].max()
        total_qty = opportunities['tradeable_qty'].sum()
        
        # Find best opportunity
        best_opp = opportunities.loc[opportunities['total_profit'].idxmax()]
        
        print(f"{rank}. ISIN: {isin}")
        print(f"   Total Theoretical Profit: €{total_profit:,.2f}")
        print(f"   Number of opportunities: {num_opps}")
        print(f"   Average profit per opportunity: €{avg_profit:,.2f}")
        print(f"   Max profit per opportunity: €{max_profit:,.2f}")
        print(f"   Total tradeable quantity: {total_qty:,.0f} shares")
        print()
        print(f"   Best Opportunity:")
        print(f"     Buy at: {best_opp['buy_exchange']} @ €{best_opp['buy_price']:.4f}")
        print(f"     Sell at: {best_opp['sell_exchange']} @ €{best_opp['sell_price']:.4f}")
        print(f"     Profit per share: €{best_opp['profit_per_share']:.4f}")
        print(f"     Quantity: {best_opp['tradeable_qty']:.0f} shares")
        print(f"     Total profit: €{best_opp['total_profit']:.2f}")
        print()
    
    # Summary table
    print("="*80)
    print("TOP 5 SUMMARY TABLE")
    print("="*80)
    summary_table = pd.DataFrame({
        'ISIN': top_isins['ISIN'].values,
        'Total Profit at 0 Latency (€)': [f"€{p:,.2f}" for p in top_isins['Profit_EUR'].values]
    })
    print(summary_table.to_string(index=False))
    print()
    
    # Sanity checks
    print("="*80)
    print("SANITY CHECKS")
    print("="*80)
    print("✓ Checking if profits are reasonable...")
    print()
    
    for _, row in top_isins.iterrows():
        isin = row['ISIN']
        
        # Reload to get average price
        data_dict_raw = loader.load_isin_data(isin)
        cleaner = DataCleaner()
        clean_data = cleaner.clean_all_venues(data_dict_raw)
        tape_builder = ConsolidatedTape(time_bin_ms=100)
        consolidated = tape_builder.create_tape(clean_data)
        signal_gen = SignalGenerator()
        opportunities = signal_gen.detect_arbitrage_opportunities(consolidated)
        opportunities = signal_gen.apply_rising_edge(opportunities)
        opportunities = opportunities[opportunities.get('is_rising_edge', False)].copy()
        
        if opportunities.empty:
            continue
        
        avg_price = (opportunities['buy_price'].mean() + opportunities['sell_price'].mean()) / 2
        avg_profit_per_share = opportunities['profit_per_share'].mean()
        avg_profit_pct = (avg_profit_per_share / avg_price) * 100
        
        print(f"{isin}:")
        print(f"  Average price: €{avg_price:.4f}")
        print(f"  Average profit per share: €{avg_profit_per_share:.4f}")
        print(f"  Average profit %: {avg_profit_pct:.4f}%")
        
        if avg_profit_pct < 1.0:
            print(f"  ✓ Profit percentage looks reasonable (<1%)")
        else:
            print(f"  ⚠ Profit percentage seems high (>1%)")
        print()
else:
    print("No data available for Top Opportunities.")


No data available for Top Opportunities.


## Summary & Answers to Key Questions

Respuestas a las 3 preguntas clave (igual que el otro código)


In [33]:
# ============================================================================
# SUMMARY & ANSWERS TO KEY QUESTIONS
# ============================================================================
# Igual que el otro código

if money_table_data:
    money_df = pd.DataFrame(money_table_data)
    
    # Calculate totals
    totals_by_latency = money_df.groupby('Latency_us')['Profit_EUR'].sum()
    
    max_profit = totals_by_latency.get(0, 0)
    profit_1ms = totals_by_latency.get(1000, 0)
    profit_10ms = totals_by_latency.get(10000, 0)
    profit_100ms = totals_by_latency.get(100000, 0)
    
    # Count ISINs with opportunities
    isins_with_opps = money_df[money_df['Latency_us'] == 0]
    isins_with_opps = isins_with_opps[isins_with_opps['Profit_EUR'] > 0]
    num_isins = len(isins_with_opps)
    
    # Calculate half-life (50% profit remaining)
    half_profit = max_profit / 2
    half_life_ms = None
    
    for lat_us in LATENCY_LEVELS:
        profit = totals_by_latency.get(lat_us, 0)
        if profit <= half_profit:
            half_life_ms = lat_us / 1000
            break
    
    if half_life_ms is None:
        half_life_ms = 100.0  # Default if not reached
    
    # Check for losses
    has_losses = any(p < 0 for p in totals_by_latency.values)
    
    print("="*80)
    print("ANSWERS TO KEY QUESTIONS")
    print("="*80)
    print()
    
    print("1. Do arbitrage opportunities still exist in Spanish equities?")
    if max_profit > 0:
        print(f"   ✓ YES! Found arbitrage opportunities with total theoretical profit of €{max_profit:,.2f}")
        print(f"   ✓ Number of ISINs with opportunities: {num_isins}")
    else:
        print("   ✗ NO arbitrage opportunities found.")
    print()
    
    print("2. What is the maximum theoretical profit (assuming 0 latency)?")
    print(f"   Maximum theoretical profit: €{max_profit:,.2f}")
    if num_isins > 0:
        top_isin = isins_with_opps.nlargest(1, 'Profit_EUR').iloc[0]
        print(f"   Top ISIN: {top_isin['ISIN']} with €{top_isin['Profit_EUR']:,.2f}")
    print()
    
    print("3. The 'Latency Decay' Curve: How quickly does profit vanish?")
    print(f"   At 0µs (0ms):     €{max_profit:,.2f} (100.0%)")
    if profit_1ms is not None:
        pct_1ms = (profit_1ms / max_profit * 100) if max_profit != 0 else 0
        print(f"   At 1,000µs (1ms):  €{profit_1ms:,.2f} ({pct_1ms:.1f}%)")
    if profit_10ms is not None:
        pct_10ms = (profit_10ms / max_profit * 100) if max_profit != 0 else 0
        print(f"   At 10,000µs (10ms): €{profit_10ms:,.2f} ({pct_10ms:.1f}%)")
    if profit_100ms is not None:
        pct_100ms = (profit_100ms / max_profit * 100) if max_profit != 0 else 0
        print(f"   At 100,000µs (100ms): €{profit_100ms:,.2f} ({pct_100ms:.1f}%)")
    print()
    print(f"   Half-life (50% profit remaining): ~{half_life_ms:.1f}ms")
    print()
    
    if has_losses:
        print("   ⚠ WARNING: Some latencies resulted in losses (negative profits).")
        print("     This indicates that arbitrage opportunities can turn into losses with latency.")
        print()
    
    print("="*80)
    print("ANALYSIS COMPLETE")
    print("="*80)
else:
    print("No data available for summary.")


No data available for summary.
